In [18]:
import h3
import pandas as pd
import requests
import json

### #1 Get all PALO ALTO h3_codes 

In [19]:
# Use open street map to get the coordinates of Palo Alto
# Set up connection details
url_1 = 'https://nominatim.openstreetmap.org/search.php'
params_2 = {
'q':'paloalto+USA', 
'polygon_geojson':1,
'format':'geojson'
}
#Call api
geo_object = requests.get(url_1, params_2)
# Create json object
geo_json_object = json.loads(geo_object.content)
# Extract geography polygon from nested json
gea_feature = geo_json_object.get('features')[0].get('geometry')


In [20]:
PA_h3_from_polyfill = h3.polyfill_geojson(gea_feature, res=9)

PA_h3_list =[]
for i in PA_h3_from_polyfill:
    PA_h3_list.append(i)

In [22]:
# Make DF for Palo alto for POI
PA_dict = {
    'city': 'PA',
    'h3_code': PA_h3_list
}
PA_poi_df = pd.DataFrame(PA_dict)
PA_unique_h3_code_list = PA_poi_df.h3_code.unique()

### #2 Get MUNICH h3_codes

In [23]:
url_1 = 'https://nominatim.openstreetmap.org/search.php'
params_2 = {
'q':f'munich', 
'polygon_geojson':1,
'format':'geojson'
}

geo_object = requests.get(url_1, params_2)
geo_json_object = json.loads(geo_object.content)
geo_feature = geo_json_object.get('features')[0].get('geometry')

In [24]:
# geo_feature gives back a multiPolygon
# We can't work with it so we manipulate the dict

manipulated_geo_feature = {
    'type': 'Polygon',
    'coordinates': geo_feature["coordinates"][0]
}

In [25]:
MUC_h3_from_polyfill = h3.polyfill_geojson(manipulated_geo_feature, res=9)
MUC_h3_list =[]
for i in MUC_h3_from_polyfill:
    MUC_h3_list.append(i)

    # Make DF for Munich for POI
MUC_dict = {
    'city': 'MUC',
    'h3_code': MUC_h3_list
}
MUC_poi_df = pd.DataFrame(MUC_dict)

### #3 Get PALO ALTO all POI

In [34]:
#find the center of every h3_code

PA_geo_center_list = []

for i in PA_h3_list:
    k = list(h3.h3_to_geo(i))
    PA_geo_center_list.append(k)

PA_geo_center_list_string = []

def make_string_geo():
    for i in PA_geo_center_list:
        string = str(i[0]) + ',' + str(i[1])
        PA_geo_center_list_string.append(string)
make_string_geo()

Run below Code only if needed... instead read CSV "PA_poi_df.csv"

In [35]:


# api_key= 'Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0'
# # url_range = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=700&limit=100&apiKey={api_key}'

# PA_title_list = []
# PA_cat_id_list = []
# PA_cat_def_list = []
# PA_cat_name_list = []
# PA_lat_list = []
# PA_lng_list = []
# PA_source_geo = []
# PA_h3_source_list = []


# count = 0

# for index, GEO in enumerate(PA_geo_center_list_string):
#     resp = requests.get(url = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=200&limit=100&apiKey={api_key}')
#     data= resp.json()

#     for i in data['items']:
#         if 'categories' not in i:
#             continue
#         else:
#             PA_title_list.append(str(i['title']))
#             PA_cat_id_list.append(str(i['categories'][0]['id']))
#             PA_cat_name_list.append(str(i['categories'][0]['name']))
#             PA_lat_list.append(str(i['position']['lat']))
#             PA_lng_list.append(str(i['position']['lng']))
#             PA_source_geo.append(GEO)
#             PA_h3_source_list.append(PA_h3_list[index])
#             count +=1
# print('POI`s found: ' + str(count))

POI`s found: 4395


In [36]:
PA_here_dict = { 'title': PA_title_list, 
                'cat_id': PA_cat_id_list, 
                'cat_name': PA_cat_name_list, 
                'h3_source': PA_h3_source_list} 
PA_poi_df = pd.DataFrame(PA_here_dict)

In [39]:
def make_cat_def(row):
    if row['cat_id'][:1] == '1':
        return 'Eat & Drink'
    elif row['cat_id'][:1] == '2':
        return 'Going Out-Entertainment'
    elif row['cat_id'][:1] == '3':
        return 'Sights and Museums'
    elif row['cat_id'][:1] == '4':
        return 'Transport'
    elif row['cat_id'][:1] == '5':
        return 'Accommodations'
    elif row['cat_id'][:1] == '6':
        return 'Shopping'
    elif row['cat_id'][:1] == '7':
        return 'Business and Services'
    elif row['cat_id'][:1] == '8':
        return 'Facilities'
    elif row['cat_id'][:1] == '9':
        return 'Areas and Buildings'

PA_poi_df['cat_definition'] = PA_poi_df.apply(lambda row : make_cat_def(row), axis=1)

In [40]:
PA_poi_df = PA_poi_df[[     'h3_source',
                            'cat_definition', 
                            'cat_name',
                            'title'
                ]]

In [41]:
PA_poi_df.to_csv('PA_poi_df.csv', sep=',')

### #4 Get MUNICH all POI

In [42]:
#find the center of every h3_code

MUC_geo_center_list = []

for i in MUC_h3_list:
    k = list(h3.h3_to_geo(i))
    MUC_geo_center_list.append(k)

MUC_geo_center_list_string = []

def make_string_geo():
    for i in MUC_geo_center_list:
        string = str(i[0]) + ',' + str(i[1])
        MUC_geo_center_list_string.append(string)
make_string_geo()

In [43]:
api_key= 'Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0'
# url_range = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=700&limit=100&apiKey={api_key}'

MUC_title_list = []
MUC_cat_id_list = []
MUC_cat_name_list = []
MUC_h3_source_list = []


count = 0

for index, GEO in enumerate(MUC_geo_center_list_string):
    resp = requests.get(url = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=200&limit=100&apiKey={api_key}')
    data= resp.json()

    for i in data['items']:
        if 'categories' not in i:
            continue
        else:
            MUC_title_list.append(str(i['title']))
            MUC_cat_id_list.append(str(i['categories'][0]['id']))
            MUC_cat_name_list.append(str(i['categories'][0]['name']))
            MUC_h3_source_list.append(MUC_h3_list[index])
            count +=1
print('POI`s found: ' + str(count))

POI`s found: 89524


In [44]:
MUC_here_dict = { 'title': MUC_title_list, 
                'cat_id': MUC_cat_id_list, 
                'cat_name': MUC_cat_name_list, 
                'h3_source': MUC_h3_source_list} 
MUC_poi_df = pd.DataFrame(MUC_here_dict)

In [45]:
MUC_poi_df['cat_definition'] = MUC_poi_df.apply(lambda row : make_cat_def(row), axis=1)

In [46]:
MUC_poi_df = MUC_poi_df[[       'h3_source',
                                'cat_definition', 
                                'cat_name',
                                'title'
                ]]

In [47]:
MUC_poi_df.to_csv('MUC_poi_df.csv', sep=',')